In [1]:
%%bash

# Create a BigQuery dataset for babyweight if it doesn't exist
datasetexists=$(bq ls -d | grep -w forecast)

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset already exists!"

else
    echo "Creating BigQuery dataset titled: forecast"
    
    bq --location=US mk --dataset \
        --description "chicago taxi trip data" \
        $PROJECT:forecast
    echo "Here are your current datasets:"
    bq ls
fi

Creating BigQuery dataset titled: forecast
Dataset 'gcp-poc-346506:forecast' successfully created.
Here are your current datasets:
        datasetId         
 ------------------------ 
  chicagotaxitrips        
  forecast                
  nyc_bike_share          
  timeseries              
  timeseries_forecasting  
  vertex_automl1          


In [7]:
%%bigquery 

SELECT
    *
FROM
    `bigquery-public-data.iowa_liquor_sales.sales`
LIMIT 20

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,...,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,S06314200010,2012-06-28,2567,Hy-Vee Drugstore / Davenport,2200 WEST KIMBERLY,DAVENPORT,52806,POINT (-90.608201 41.560663),82,Scott,...,36908,Mccormick Vodka Pet,6,1750,7.46,11.19,60,671.40,105.00,27.74
1,S16437900117,2013-12-21,2651,Hy-Vee / Waverly,1311 4 STREET SW,WAVERLY,50677,POINT (-92.475403 42.712263),9,Bremer,...,67527,Kahlua Coffee Liqueur,12,1000,15.25,22.87,4,91.48,4.00,1.06
2,INV-30460400026,2020-09-23,2190,"Central City Liquor, Inc.",1460 2ND AVE,Des Moines,50314.0,POINT (-93.619787 41.60566),77,POLK,...,67524,Kahlua Coffee,24,375,6.49,9.74,1,9.74,0.37,0.09
3,INV-30501200139,2020-09-24,4129,Cyclone Liquors,626 Lincoln Way,Ames,50010.0,POINT (-93.618911 42.022854),85,STORY,...,67527,Kahlua Coffee,12,1000,15.15,22.73,2,45.46,2.00,0.52
4,S11176500041,2013-03-19,2607,Hy-Vee Wine and Spirits / Shenandoah,520 SO FREMONT,SHENANDOAH,51601,POINT (-95.385111 40.761736),73,Page,...,54444,Paramount Apricot Flavored Brandy,24,375,3.54,5.31,5,26.55,1.88,0.50
5,S28047700025,2015-09-22,3056,Clarion Super Valu Ii,325 CENTRAL AVE W,CLARION,50525,POINT (-93.735875 42.731643),99,Wright,...,55087,Paramount Blackberry Brandy,12,1000,6.53,9.80,4,39.20,4.00,1.06
6,INV-22549300037,2019-10-15,2651,Hy-Vee / Waverly,1311 4 Street SW,Waverly,50677,POINT (-92.475403 42.712263),9,BREMER,...,33717,Paramount Sloe Gin,12,1000,5.42,8.13,2,16.26,2.00,0.52
7,INV-22572600008,2019-10-16,2673,Hy-Vee # 6/ Des Moines,420 Court Ave,Des Moines,50309,POINT (-93.62245400000002 41.584842),77,POLK,...,27025,Bulleit 95 Rye,12,750,16.50,24.75,120,2970.00,90.00,23.77
8,INV-18805600001,2019-04-16,2623,Hy-Vee Food Store #4 / Sioux City,2827 Hamilton Blvd,Sioux City,51104,POINT (-96.417783 42.519886),97,WOODBURY,...,77487,Tortilla Gold Dss,12,1000,4.85,7.28,120,873.60,120.00,31.70
9,INV-18911400005,2019-04-22,5241,BP / Dubuque,"1450, Loras Blvd",Dubuque,52001,POINT (-90.687255 42.500724),48,Iowa,...,67522,Kahlua Coffee Mini,12,500,6.60,9.90,4,39.60,2.00,0.52


In [9]:
%%bigquery

SELECT
    date,
    SUM(volume_sold_liters) total_sold_liters
FROM
    `bigquery-public-data.iowa_liquor_sales.sales`
WHERE
    EXTRACT (year from date) = 2020 OR EXTRACT (year from date) = 2021
    GROUP BY date
LIMIT 10;

,date,total_sold_liters
0,2020-09-25,97184.65
1,2021-10-12,111222.15
2,2021-07-07,145426.53
3,2021-04-06,85030.68
4,2020-02-03,89810.99
5,2020-04-21,100685.13
6,2021-07-12,73837.71
7,2021-07-16,72537.48
8,2021-07-15,76630.82
9,2020-10-02,96777.36


In [2]:
%%bigquery

CREATE TABLE `gcp-poc-346506.forecast.wines` as
SELECT
    date,
    SUM(volume_sold_liters) total_sold_liters
FROM
    `bigquery-public-data.iowa_liquor_sales.sales`
WHERE
    EXTRACT (year from date) = 2020 OR EXTRACT (year from date) = 2021
    GROUP BY date;

""


In [3]:
%%bigquery

CREATE OR REPLACE MODEL `gcp-poc-346506.forecast.wine_model`
OPTIONS
    (
        model_type = 'ARIMA',
        time_series_timestamp_col = 'date',
        time_series_data_col = 'total_sold_liters',
        auto_arima = TRUE,
        data_frequency = 'AUTO_FREQUENCY'
    ) 
AS 
SELECT * FROM `gcp-poc-346506.forecast.wines`;

""


In [4]:
%%bigquery

SELECT
    *
FROM
    ML.TRAINING_INFO(MODEL `gcp-poc-346506.forecast.wine_model`)

,training_run,iteration,duration_ms
0,0,0,3778


In [6]:
%%bigquery

SELECT * FROM ML.EVALUATE(MODEL `gcp-poc-346506.forecast.wine_model`);

,non_seasonal_p,non_seasonal_d,non_seasonal_q,has_drift,log_likelihood,AIC,variance,seasonal_periods
0,0,1,5,False,-7933.962921,15879.925841,1.649923e+08,[WEEKLY]
1,0,1,5,True,-7934.108060,15882.216119,1.650628e+08,[WEEKLY]
2,3,1,1,False,-7954.162913,15918.325825,1.752915e+08,[WEEKLY]
3,1,1,4,True,-7952.991356,15919.982713,1.747242e+08,[WEEKLY]
4,3,1,1,True,-7954.363669,15920.727337,1.753820e+08,[WEEKLY]
5,2,1,3,False,-7954.966604,15921.933209,1.756744e+08,[WEEKLY]
6,3,1,2,False,-7955.120341,15922.240682,1.757609e+08,[WEEKLY]
7,2,1,3,True,-7955.053921,15924.107842,1.757267e+08,[WEEKLY]
8,3,1,2,True,-7955.402447,15924.804893,1.759133e+08,[WEEKLY]
9,1,1,1,False,-7960.083049,15926.166098,1.781880e+08,[WEEKLY]


In [5]:
%%bigquery

SELECT * FROM
ML.FORECAST(MODEL `gcp-poc-346506.forecast.wine_model`, 
STRUCT(30 AS horizon, 0.8 AS confidence_level));

,forecast_timestamp,forecast_value,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,confidence_interval_lower_bound,confidence_interval_upper_bound
0,2022-01-01 00:00:00+00:00,33565.875370,12844.931924,0.8,17094.990721,50036.760019,17094.990721,50036.760019
1,2022-01-02 00:00:00+00:00,54773.934824,13442.213389,0.8,37537.164129,72010.705519,37537.164129,72010.705519
2,2022-01-03 00:00:00+00:00,80323.985412,13473.597749,0.8,63046.970971,97600.999852,63046.970971,97600.999852
3,2022-01-04 00:00:00+00:00,96847.479045,15438.558595,0.8,77050.821556,116644.136534,77050.821556,116644.136534
4,2022-01-05 00:00:00+00:00,107304.657303,17901.053014,0.8,84350.376111,130258.938495,84350.376111,130258.938495
5,2022-01-06 00:00:00+00:00,103771.987969,17917.516123,0.8,80796.596351,126747.379588,80796.596351,126747.379588
6,2022-01-07 00:00:00+00:00,95949.283244,17933.964120,0.8,72952.800579,118945.765909,72952.800579,118945.765909
7,2022-01-08 00:00:00+00:00,42045.760141,17950.397046,0.8,19028.205755,65063.314527,19028.205755,65063.314527
8,2022-01-09 00:00:00+00:00,60803.931539,17966.814941,0.8,37765.324705,83842.538373,37765.324705,83842.538373
9,2022-01-10 00:00:00+00:00,88179.063493,17983.217848,0.8,65119.423430,111238.703555,65119.423430,111238.703555
